In [40]:
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud, STOPWORDS
import warnings
warnings.filterwarnings("ignore")

### Item-To-Item Filtering model

In [65]:
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
Ratings=pd.read_csv("../input/ratings.csv",encoding="ISO-8859-1")
Movies=pd.read_csv("../input/movies.csv",encoding="ISO-8859-1")
Tags=pd.read_csv("../input/tags.csv",encoding="ISO-8859-1")

In [67]:
movie_rating_count = Ratings.groupby('movieId')['rating'].count()
movie_rating_count = pd.DataFrame(movie_rating_count)
movie_rating_count.columns = ['rating_count']
movie_rating_count['movieId'] = movie_rating_count.index
movie_rating_count.reset_index(drop=True)
Ratings = Ratings.merge(movie_rating_count,right_on='movieId',left_on='movieId')

In [68]:
RatingMat = Ratings.pivot_table(index=['userId'],columns=['movieId'],values=['rating'],fill_value=0)

Original_RatingMat = RatingMat.copy()
RatingMat.head()

rating                       ...                              
movieId 1      2      3      4       ...   193583 193585 193587 193609
userId                               ...                              
1          4.0    0.0    4.0    0.0  ...      0.0    0.0    0.0      0
2          0.0    0.0    0.0    0.0  ...      0.0    0.0    0.0      0
3          0.0    0.0    0.0    0.0  ...      0.0    0.0    0.0      0
4          0.0    0.0    0.0    0.0  ...      0.0    0.0    0.0      0
5          4.0    0.0    0.0    0.0  ...      0.0    0.0    0.0      0

[5 rows x 9724 columns]

In [69]:
RatingMat = RatingMat.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

RatingMat.columns = RatingMat.columns.droplevel()
RatingMat.head(5)

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.779165,-0.020835,0.779165,-0.020835,-0.020835,0.779165,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,...,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835
2,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,...,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355,-0.002355
3,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,0.098046,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,...,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954,-0.001954
4,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,0.584204,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,0.384204,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,...,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796,-0.015796
5,0.796709,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,0.796709,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-0.003291,-

In [70]:
user_similarity = cosine_similarity(RatingMat)
user_sim_df = pd.DataFrame(user_similarity,index=RatingMat.index,columns=RatingMat.index)
user_sim_df.head(5)

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.019400,0.053056,0.176920,0.120866,0.104418,0.143793,0.128547,0.055268,-0.000298,0.122323,0.007960,0.085803,0.104457,0.146283,0.157364,0.252953,0.187725,0.300040,0.141389,0.126253,0.036855,0.091566,0.141627,0.092625,0.100118,0.225827,0.174090,0.133656,0.092081,0.156080,0.133226,0.135969,0.071351,0.068152,0.061368,0.139595,0.127586,0.320120,0.081012,...,0.076683,0.114652,0.233626,0.087359,0.054737,0.046064,0.304202,-0.007817,0.130296,0.225740,0.048369,0.038085,0.041564,0.088216,0.103667,0.103559,0.105562,0.175445,0.110124,0.298475,0.127919,0.131652,0.162223,0.122497,0.105116,0.096131,0.291054,0.004857,0.241534,0.263481,0.066256,0.149942,0.186978,0.056530,0.134412,0.121981,0.254200,0.262241,0.085434,0.098719
2,0.019400,1.000000,-0.002594,-0.003804,0.013183,0.016257,0.021567,0.023750,-0.003448,0.061880,0.040383,-0.003041,0.041117,0.013346,0.114666,0.088977,0.098900,0.158508,-0.001030,0.006179,0.081791,0.140386,0.007765,0.125141,0.223866,-0.002451,-0.006093,0.046978,0.099785,0.146642,-0.003736,0.012534,0.026120,0.038819,-0.002552,0.050279,0.026132,0.018681,-0.005285,0.012210,...,-0.005245,0.026354,0.161412,0.033077,-0.003059,-0.002230,0.008246,-0.002727,-0.004535,0.052241,0.133752,0.216222,0.039837,-0.004805,0.123907,0.082799,-0.006863,0.052246,0.023167,0.043761,0.032507,0.025763,0.057158,0.005373,-0.002390,0.095872,0.000955,0.046332,0.082509,0.009476,0.198549,0.010888,-0.004030,-0.005345,-0.007919,0.011299,0.005813,0.032730,0.024373,0.089329
3,0.053056,-0.002594,1.000000,-0.004556,0.001887,-0.004577,-0.005634,0.001703,-0.003111,-0.005501,-0.003824,-0.002743,-0.002644,-0.000189,0.011930,0.027669,0.004859,0.017891,0.007140,0.001523,-0.005974,-0.001527,-0.002251,-0.002509,0.002615,-0.002212,-0.003148,0.019006,0.001631,-0.002847,0.000150,0.010729,0.001264,-0.003509,-0.002303,-0.003570,-0.002200,0.000081,-0.001108,-0.002279,...,0.016291,0.002158,-0.008390,-0.002296,-0.002760,0.027879,0.034155,-0.002460,0.001685,0.023832,0.005497,-0.003632,-0.003577,-0.004335,-0.003810,-0.006082,0.009749,0.000437,0.001782,0.015148,-0.003412,-0.004604,0.001751,-0.005133,-0.002156,0.016483,0.019437,-0.002066,0.018550,0.000416,0.000150,-0.000585,0.011211,-0.004822,0.003678,-0.003246,0.012885,0.008096,-0.002963,0.015962
4,0.176920,-0.003804,-0.004556,1.000000,0.121018,0.065719,0.100602,0.054235,0.002417,0.015615,0.044320,0.042513,0.070111,0.040059,0.056804,0.153127,0.132635,0.095222,0.178130,0.094761,0.025259,0.030562,0.079852,0.057863,0.025941,0.056055,0.115024,0.106839,0.049399,0.050508,0.117905,0.189416,0.152799,0.033798,0.043981,0.104827,0.055277,0.079278,0.230626,0.040150,...,0.005013,0.168649,0.104275,0.058571,0.053684,0.000124,0.154665,-0.007181,0.088171,0.123962,0.035373,0.037623,0.038646,0.029166,0.078319,0.044753,0.123212,0.039545,0.056425,0.190823,0.107595,0.055743,0.206465,0.063496,0.086684,0.082179,0.254500,0.010196,0.154673,0.183269,0.072848,0.114287,0.281866,0.039699,0.065493,0.164831,0.115118,0.116861,0.023930,0.062523
5,0.120866,0.013183,0.001887,0.121018,1.000000,0.294138,0.101725,0.426576,-0.004185,0.023471,0.179624,0.055388,0.013661,0.218294,0.103749,0.076257,0.157100,0.109778,0.085258,0.087905,0.046076,0.027086,0.060235,0.090070,0.037485,0.292237,0.093956,0.109967,0.063418,0.061957,0.064362,0.229066,0.277044,0.024416,0.309387,0.017368,0.301073,0.374389,0.147607,0.316861,...,0.013787,0.117988,0.120258,0.339365,0.045777,-0.002707,0.115218,-0.003310,0.177915,0.106447,0.053618,-0.004887,0.120437,0.344517,0.024320,0.064117,0.147572,0.374333,0.227833,0.141074,-0.004591,0.355790,0.166667,0.130174,0.070564,0.085002,0.105146,-0.002780,0.076954,0.124588,0.061912,0.414931

In [133]:
Rating_Movie = Ratings.merge(Movies, left_on='movieId',right_on='movieId')
def predict_movies_item_to_item(user_id, num_of_movies):
    curr_user_rated_movie = Ratings[(Ratings.userId == user_id) & (Ratings.rating != 0)]['movieId']
    curr_user_rated_movie = pd.DataFrame(curr_user_rated_movie,columns=['movieId'])
    curr_user_similarity = pd.DataFrame(user_sim_df.loc[user_id])
    curr_user_similarity.rename(columns={user_id:'similarity'},inplace=True)
    curr_user_similarity.head()
    RatingMatTranspose = RatingMat.T
    RatingMatTranspose = RatingMatTranspose.loc[~RatingMatTranspose.index.isin(curr_user_rated_movie.movieId.tolist())]
    try:
        Original_RatingMat.columns = Original_RatingMat.columns.droplevel()
    except AttributeError:
        pass
    Weighed_avg = []

    for movieId in RatingMatTranspose.index:
        user_not_rated = Original_RatingMat[Original_RatingMat[movieId] == 0].index
        Total_weight = np.sum(curr_user_similarity.loc[~curr_user_similarity.index.isin(user_not_rated.tolist())]['similarity'])
        Weighed_avg.append(np.dot(RatingMatTranspose.loc[movieId],curr_user_similarity.similarity) / (Total_weight))
    
    Weighed_avg = pd.DataFrame(Weighed_avg,columns=['weighted_avg'])
    Weighed_avg.index = RatingMatTranspose.index
    top_recommendation = Weighed_avg.sort_values(by='weighted_avg',ascending=False)
    Watched_Genre = pd.DataFrame(Rating_Movie[Rating_Movie.userId==user_id].groupby('genres').size(),columns=['Count'])
    top_recommendation['movieId'] = top_recommendation.index
    Movies_recommended = Movies.merge(top_recommendation, left_on='movieId',right_on='movieId')
    Watched_recommended_Genre = pd.DataFrame(Movies_recommended.groupby('genres').size(),columns=['Count'])
    
    item_similarity = cosine_similarity(RatingMat.T)
    item_sim_df = pd.DataFrame(item_similarity,index=RatingMat.columns,columns=RatingMat.columns)
    item_sim_df.head(5)
    
    curr_movie_similarity = pd.DataFrame(item_sim_df.loc[5])
    curr_movie_similarity.rename(columns={5:'similarity'},inplace=True)
    curr_movie_similarity.head()
    
    Weighed_movie_avg = []
 
    for movieId in item_sim_df.index:
        curr_movie_similarity = pd.DataFrame(item_sim_df.loc[movieId])
        Total_movie_weight = np.sum(item_sim_df.loc[movieId])
        Weighed_movie_avg.append(np.dot(RatingMat.loc[user_id],curr_movie_similarity) \
                                 /(np.abs(Total_movie_weight)))
        
    Weighed_movie_avg = pd.DataFrame(Weighed_movie_avg)
    Weighed_movie_avg.index = RatingMat.columns
    Weighed_movie_avg.columns = ['weighted_avg']
    Weighed_movie_avg.head()
    
    top_recommendation_itemwise = Weighed_movie_avg.sort_values(by='weighted_avg',ascending=False).head(num_of_movies)
    
    return top_recommendation_itemwise.index

In [72]:
top_5_recommendation = predict_movies_item_to_item(1, 5)

In [76]:
for i in range(Movies.shape[0]):
    for x in top_5_recommendation:
        if Movies.movieId[i] == x:
            print(Movies.iloc[i])

movieId                                     940
title      Adventures of Robin Hood, The (1938)
genres                 Action|Adventure|Romance
Name: 720, dtype: object
movieId                     2454
title            Fly, The (1958)
genres     Horror|Mystery|Sci-Fi
Name: 1845, dtype: object
movieId                    2922
title      Hang 'Em High (1968)
genres      Crime|Drama|Western
Name: 2198, dtype: object
movieId                               45028
title      Prairie Home Companion, A (2006)
genres                 Comedy|Drama|Musical
Name: 6192, dtype: object
movieId                           47950
title              Hollywoodland (2006)
genres     Crime|Drama|Mystery|Thriller
Name: 6289, dtype: object


### User-to-User Filtering model

In [41]:
Ratings=pd.read_csv("../input/ratings.csv",encoding="ISO-8859-1")
Movies=pd.read_csv("../input/movies.csv",encoding="ISO-8859-1")
Tags=pd.read_csv("../input/tags.csv",encoding="ISO-8859-1")

In [42]:
Mean = Ratings.groupby(by="userId",as_index=False)['rating'].mean()
Rating_avg = pd.merge(Ratings,Mean,on='userId')
Rating_avg['adg_rating']=abs(Rating_avg['rating_x']-Rating_avg['rating_y'])
Rating_avg.head()

,userId,movieId,rating_x,timestamp,rating_y,adg_rating
0,1,1,4.0,964982703,4.366379,0.366379
1,1,3,4.0,964981247,4.366379,0.366379
2,1,6,4.0,964982224,4.366379,0.366379
3,1,47,5.0,964983815,4.366379,0.633621
4,1,50,5.0,964982931,4.366379,0.633621


In [43]:
from sklearn.metrics.pairwise import cosine_similarity
final=pd.pivot_table(Rating_avg,values='adg_rating',index='userId',columns='movieId')

In [ ]:
final.head()

In [44]:
final_movie = final.fillna(0)

In [ ]:
final_movie.head()

In [45]:
cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine, 0)
similarity_with_movie =pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_movie.index
similarity_with_movie.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.001265,0.047517,0.142758,0.102018,0.127954,0.103623,0.091396,0.044187,0.051122,0.076418,0.013266,0.059383,0.062815,0.088682,0.138799,0.158020,0.141620,0.262479,0.100536,0.095223,0.027701,0.070757,0.095915,0.052090,0.080614,0.167029,0.111445,0.083619,0.048763,0.079248,0.132709,0.095761,0.056186,0.061370,0.048596,0.094555,0.124339,0.229017,0.087239,...,0.077469,0.107611,0.135573,0.071848,0.058927,0.024133,0.225602,0.0,0.152127,0.135486,0.040620,0.011207,0.029729,0.084260,0.074541,0.075344,0.085522,0.154883,0.096272,0.235135,0.094326,0.109083,0.137956,0.045891,0.048003,0.049375,0.171888,0.022298,0.204735,0.184128,0.054995,0.125598,0.164904,0.063877,0.139467,0.125304,0.234992,0.195194,0.077982,0.091369
2,0.001265,0.000000,0.000000,0.017164,0.021796,0.023480,0.011114,0.048085,0.000000,0.110242,0.010385,0.000000,0.003048,0.012518,0.121918,0.051692,0.077995,0.095250,0.000494,0.000610,0.019189,0.057633,0.001169,0.072575,0.041679,0.000000,0.000000,0.013309,0.024525,0.057555,0.000000,0.032900,0.015727,0.011875,0.000000,0.046177,0.007806,0.042923,0.000000,0.026103,...,0.000000,0.023915,0.079271,0.053313,0.000000,0.000000,0.029436,0.0,0.000000,0.024679,0.073312,0.058553,0.023075,0.000000,0.031813,0.036558,0.000000,0.050392,0.035668,0.026260,0.002483,0.010316,0.029937,0.000338,0.000000,0.041712,0.006005,0.001918,0.051916,0.006663,0.089434,0.031581,0.002025,0.000000,0.000000,0.006542,0.020504,0.019398,0.060091,0.048396
3,0.047517,0.000000,0.000000,0.011260,0.031539,0.014908,0.000000,0.032471,0.000000,0.000000,0.000000,0.000000,0.000000,0.008453,0.046046,0.025305,0.028968,0.029523,0.018386,0.030962,0.022848,0.026876,0.030150,0.009327,0.015357,0.000000,0.005516,0.037464,0.024492,0.000000,0.001705,0.045848,0.030532,0.029048,0.000000,0.000000,0.000000,0.028985,0.028645,0.003331,...,0.025425,0.034906,0.000000,0.000000,0.000000,0.054952,0.053956,0.0,0.036996,0.015786,0.037817,0.000000,0.000000,0.000000,0.000000,0.031381,0.013502,0.014251,0.024086,0.044593,0.000000,0.000000,0.037259,0.007823,0.000000,0.029372,0.035092,0.000000,0.056485,0.036448,0.033022,0.026534,0.043068,0.000000,0.001431,0.041202,0.043654,0.018478,0.000000,0.028409
4,0.142758,0.017164,0.011260,0.000000,0.094994,0.085745,0.088957,0.058842,0.005874,0.088482,0.086375,0.034050,0.080550,0.089435,0.061926,0.161349,0.137858,0.102874,0.171669,0.053482,0.073164,0.049568,0.059317,0.060862,0.024168,0.099039,0.117409,0.097250,0.067387,0.027831,0.084653,0.184591,0.152013,0.074765,0.019594,0.095906,0.047147,0.065339,0.185272,0.064348,...,0.005154,0.127885,0.099657,0.072262,0.072935,0.025949,0.121871,0.0,0.089232,0.169157,0.073791,0.028046,0.033928,0.038162,0.080819,0.061719,0.070131,0.066006,0.034895,0.162727,0.130094,0.071074,0.170530,0.037867,0.112045,0.042855,0.123126,0.076075,0.155810,0.160191,0.105068,0.165896,0.233322,0.069222,0.068452,0.148970,0.130854,0.121548,0.058381,0.087530
5,0.102018,0.021796,0.031539,0.094994,0.000000,0.257693,0.032843,0.374558,0.000000,0.038993,0.077486,0.066440,0.011719,0.249361,0.089324,0.060394,0.104256,0.106785,0.070066,0.059356,0.014759,0.022540,0.039401,0.048173,0.021465,0.167198,0.039423,0.074545,0.041657,0.050865,0.029947,0.191696,0.116853,0.077051,0.451739,0.017593,0.342373,0.375615,0.074737,0.195887,...,0.018846,0.048524,0.067342,0.236643,0.053900,0.000000,0.148963,0.0,0.117233,0.067417,0.064605,0.000000,0.124628,0.305678,0.030893,0.077565,0.111509,0.268236,0.153078,0.106720,0.000000,0.230558,0.092470,0.064915,0.033903,0.038965,0.063590,0.000000,0.058634,0.077285,0.051182,0.341861,0.080928,0.199155,0.092935,0.069552,0.111137,0.079205,0.222108,0.042926


In [46]:
def find_n_similar_users(similarity_with_movie, n):
    size = similarity_with_movie.shape[0]
    user_dict = dict()
    
    for i in range(size): 
        arr = similarity_with_movie.values[i]
        arr = list(enumerate(arr))
        arr.sort(key=lambda pair: pair[1])
        arr = arr[:n]
        tmp_arr = list()
        for j in arr:
            tmp_arr.append(j[0])
        user_dict[i+1] = tmp_arr
    
    df = pd.DataFrame(user_dict)
    return df.T

In [47]:
user_ = find_n_similar_users(similarity_with_movie, 30)

In [ ]:
user_

In [77]:
from collections import defaultdict

def predict_movie(user_id, n_movies):
    user_ = find_n_similar_users(similarity_with_movie, 30)
    neighbors_ids = (user_.values[user_id - 1])
    size_raw = Ratings.shape[0]
    size_col = Ratings.shape[1]
   # print(neighbors_ids)
    neigh_dict = defaultdict(list)
    for i in range(size_raw):
        for x in neighbors_ids:
            if user_id == x:
                continue
            if Ratings.userId[i] == x:
               # print(x, neigh_dict[x])
                neigh_dict[x].append((Ratings.iloc[i][1], Ratings.iloc[i][2]))
                
    for i in neigh_dict.items():
        arr = i[1]
        arr.sort(key=lambda pair: pair[1])
        arr = arr[len(arr)-5:len(arr)]
        neigh_dict[i[0]] = arr

    final_dict = defaultdict(float)
    for i in neigh_dict.values():
        for x in i:
            final_dict[x[0]]+=x[1]/5

    m_list = []
    for key, value in sorted(final_dict.items(), key=lambda item: item[1]):
        m_list.append(int(key))
    
    return list(m_list)[len(m_list)- n_movies:len(m_list)]


                

In [80]:
movies_ids = predict_movie(1, 5)

In [81]:
movies_ids

[78499, 1, 1287, 1387, 293]

In [82]:
for i in range(Movies.shape[0]):
    for x in movies_ids:
        if Movies.movieId[i] == x:
            print(Movies.iloc[i])

movieId                                              1
title                                 Toy Story (1995)
genres     Adventure|Animation|Children|Comedy|Fantasy
Name: 0, dtype: object
movieId                                                  293
title      LÃ©on: The Professional (a.k.a. The Profession...
genres                           Action|Crime|Drama|Thriller
Name: 254, dtype: object
movieId                      1287
title              Ben-Hur (1959)
genres     Action|Adventure|Drama
Name: 986, dtype: object
movieId             1387
title        Jaws (1975)
genres     Action|Horror
Name: 1067, dtype: object
movieId                                               78499
title                                    Toy Story 3 (2010)
genres     Adventure|Animation|Children|Comedy|Fantasy|IMAX
Name: 7355, dtype: object


In [128]:
import random

def find_user_movie_rat(user_id, movie_id):
    result = None
    for i in range(Ratings.shape[0]):
        if user_id == Ratings.userId[i] and movie_id == Ratings.movieId[i]:
            result = Ratings.rating[i]
            break
    return result

def calculate_recal(n_users, n_movies, function):
    users_list = []
    
    for i in range(n_users):
        users_list.append(random.choice (Ratings.userId))
        
    mean_recal = []
    for i in users_list:
        predicted_ids = function(i, n_movies)
        recal = 0
        for x in predicted_ids:
            tmp_res = find_user_movie_rat(i, x)
            if tmp_res is None:
                continue
            if tmp_res >= 3.6:
                recal+=1
            print(f'user_id = {i}, movie_id= {x}, rating in table= {tmp_res}')
        mean_recal.append(recal/n_movies)
    
    return np.mean(mean_recal)       
        

In [129]:
%%time
a = calculate_recal(10, 5, predict_movie)

user_id = 288, movie_id= 1387, rating in table= 5.0
user_id = 288, movie_id= 318, rating in table= 5.0
user_id = 380, movie_id= 32, rating in table= 5.0
user_id = 380, movie_id= 318, rating in table= 3.0
user_id = 380, movie_id= 293, rating in table= 5.0
user_id = 380, movie_id= 104, rating in table= 3.0
user_id = 474, movie_id= 4226, rating in table= 5.0
user_id = 474, movie_id= 7361, rating in table= 4.5
user_id = 474, movie_id= 1387, rating in table= 4.5
user_id = 474, movie_id= 293, rating in table= 3.5
user_id = 474, movie_id= 318, rating in table= 5.0
user_id = 274, movie_id= 318, rating in table= 4.5
user_id = 274, movie_id= 1387, rating in table= 3.5
user_id = 274, movie_id= 377, rating in table= 3.5
user_id = 274, movie_id= 293, rating in table= 4.5
user_id = 181, movie_id= 62, rating in table= 4.0
user_id = 181, movie_id= 454, rating in table= 5.0
user_id = 181, movie_id= 377, rating in table= 3.0
user_id = 322, movie_id= 356, rating in table= 3.5
user_id = 346, movie_id= 858

In [130]:
a

0.3

In [132]:
%%time
a = calculate_recal(10, 5, predict_movies_item_to_item)

KeyboardInterrupt: 